In [148]:
import pandas as pd
import numpy as np
from dataapi.AWS.dbutils import DBConnect
from dataapi.AWS.getb3derivatives import DI1
from finmath.brazilian_bonds.government_bonds import LTN, NTNF


class FixedIncomePortfolio(object):

    def __init__(self, bonds, ref_date, di1_connect, bond_quantities=None, bond_market_values=None,
                 adjust_for_convexity=False):

        self.bonds = bonds
        self.ref_date = ref_date
        self.di1_connect = di1_connect
        self.adjust_for_convexity = adjust_for_convexity
        self.bond_quantities = bond_quantities
        self.bond_market_values = bond_market_values
        self.portfolio_value = 0
        if self.bond_quantities is None and self.bond_market_values is None:
            msg = 'Please input the bond quantities or the bond market values.'
            raise AttributeError(msg)
        elif self.bond_quantities is not None:
            if len(self.bonds) != len(self.bond_quantities):
                msg = f"Bonds list length ({len(self.bonds)}) don't match the bond quantities length " \
                      f"({len(self.bond_quantities)})."
                raise ValueError(msg)
            else:
                self.bond_market_values = list()
                for bond, quantity in zip(self.bonds, self.bond_quantities):
                    self.portfolio_value += bond.price*quantity
                    self.bond_market_values.append(bond.price*quantity)
        elif self.bond_market_values is not None:
            if len(self.bonds) != len(self.bond_market_values):
                msg = f"Bonds list length ({len(self.bonds)}) don't match the bond market values length " \
                      f"({len(self.bond_market_values)})."
                raise ValueError(msg)
            else:
                self.bond_quantities = list()
                for bond, market_value in zip(self.bonds, self.bond_market_values):
                    self.portfolio_value += market_value
                    self.bond_quantities.append(market_value/bond.price)

        self.rate = sum([
            bond.rate * market_value / self.portfolio_value
            for bond, market_value in zip(self.bonds, self.bond_market_values)
        ])
        self.duration = sum([
            bond.mod_duration * market_value / self.portfolio_value
            for bond, market_value in zip(self.bonds, self.bond_market_values)
        ])

        self.convexity = sum([
            bond.convexity * market_value / self.portfolio_value
            for bond, market_value in zip(self.bonds, self.bond_market_values)
        ])

        self.dv01 = sum([
            bond.dv01 * market_value / self.portfolio_value
            for bond, market_value in zip(self.bonds, self.bond_market_values)
        ])

    def flat_rate_change(self, rate_change, is_bps=False):

        if is_bps:
            rate_change /= 10000

        new_market_value = 0
        for bond, quantity in zip(self.bonds, self.bond_quantities):
            new_price = bond.price_from_rate(rate=bond.rate + rate_change)
            new_market_value += new_price*quantity

        return new_market_value


class FixedIncomeHedge(DI1):

    def __init__(self, fi_portfolio, di1_connect):
        super().__init__(di1_connect)
        self.fi_portfolio = fi_portfolio
        self.ref_date = self.fi_portfolio.ref_date
        self.di_df = self.get_di1_information(self.ref_date)

    def get_di1_information(self, ref_date=None):

        ref_date = self.ref_date if ref_date is None else ref_date
        contracts = self.market_menu(self.ref_date)
        df = pd.DataFrame(index=contracts)
        df['MATURITY'] = [self.maturity(code) for code in contracts]
        df['VOLUME'] = [self.volume(code, ref_date) for code in contracts]
        df['DURATION'] = [-self.duration(code, ref_date) for code in contracts]
        df['CONVEXITY'] = [self.convexity(code, ref_date) for code in contracts]
        df['DV01'] = [-self.dv01(code, ref_date) for code in contracts]
        df['PRICE'] = [self.theoretical_price(code, ref_date) for code in contracts]
        df['IMPLIED_YIELD'] = [self.implied_yield(code, ref_date) for code in contracts]
        return df.sort_values(['MATURITY'])

    def di1_price(self, code, rate_change, ref_date=None, is_bps=False):

        ref_date = self.ref_date if ref_date is None else ref_date
        if is_bps:
            rate_change /= 10000
        initial_rate = self.implied_yield(code, ref_date)
        du = self.du2maturity(ref_date, code)
        return 100000 / ((1 + initial_rate + rate_change)**(du/252))

    def duration_hedge(self, min_contract_pct_volume=0.5):

        portfolio_dur = self.fi_portfolio.duration
        portfolio_value = self.fi_portfolio.portfolio_value
        min_volume = np.round(self.fi_portfolio.portfolio_value / 100000, 0)
        filtered_df = self.di_df[self.di_df['DURATION'] < portfolio_dur]
        filtered_df = filtered_df[filtered_df['VOLUME'] > min_volume/min_contract_pct_volume]
        last_row = filtered_df.shape[0]
        for n in reversed(range(last_row)):
            di_dur = filtered_df['DURATION'].iloc[n]
            di_pu = filtered_df['PRICE'].iloc[n]
            n_contracts = (portfolio_value*portfolio_dur)/(di_pu*di_dur)
            contract_code = filtered_df.index[n]
            if n_contracts < filtered_df['VOLUME'].iloc[n]:
                return {contract_code: np.round(n_contracts, 0)}

        return np.nan

    def duration_convexity_hedge(self, min_contract_pct_volume=0.5, min_upper_dur_distance=1):
        portfolio_dur = self.fi_portfolio.duration
        portfolio_value = self.fi_portfolio.portfolio_value
        portfolio_convex = self.fi_portfolio.convexity

        min_volume = np.round(self.fi_portfolio.portfolio_value / 100000, 0)

        filtered_df = self.di_df[self.di_df['VOLUME'] > min_volume/min_contract_pct_volume]
        lower_di = filtered_df[filtered_df['DURATION'] < portfolio_dur]
        lower_di_code = lower_di['VOLUME'].idxmax()
        lower_di = lower_di[lower_di.index == lower_di_code]
        upper_di = filtered_df[filtered_df['DURATION'] > portfolio_dur + min_upper_dur_distance]
        upper_di_code = upper_di['VOLUME'].idxmax()
        upper_di = upper_di[upper_di.index == upper_di_code]

        pu_lower = lower_di['PRICE'].values[0]
        dur_lower = lower_di['DURATION'].values[0]
        convex_lower = lower_di['CONVEXITY'].values[0]

        pu_upper = upper_di['PRICE'].values[0]
        dur_upper = upper_di['DURATION'].values[0]
        convex_upper = upper_di['CONVEXITY'].values[0]

        a = np.array([[pu_lower*dur_lower*(-1/100), pu_upper*dur_upper*(-1/100)],
                     [pu_lower*((convex_lower/2)*((1/100)**2)), pu_upper*((convex_upper/2)*((1/100)**2))]])
        b = np.array([portfolio_value*(portfolio_dur*(-1/100)), portfolio_value*(portfolio_convex/2*((1/100)**2))])
        quantities = np.linalg.inv(a) @ b
        return {lower_di_code: np.round(quantities[0], 0), upper_di_code: np.round(quantities[1], 0)}

In [149]:

path = 'D:/Pedro/OneDrive/MPE Insper/Renda Fixa/'
bond_df = pd.read_excel(f'{path}BZRFIRFM Index as of Feb 05 20211.xlsx')

ref_date = pd.to_datetime('2021-02-05')
print(f'Reference Date {ref_date}')

bond_names = list(bond_df['Description'].values)
market_values = list(bond_df['Market Value'].values)
prices = list(bond_df['Vendor Price'].values)

bond_instruments = list()
for bond_name, market_value, price in zip(bond_names, market_values, prices):
    split_name = bond_name.split(' ')
    expiry = pd.to_datetime(split_name[-1]).date()
    if split_name[0] == 'BLTN':
        bond_instruments.append(LTN(expiry=expiry,
                                    principal=1000,
                                    price=price,
                                    ref_date=ref_date))
    if split_name[0] == 'BNTNF':
        bond_instruments.append(NTNF(expiry=expiry,
                                     principal=1000,
                                     price=price,
                                     ref_date=ref_date))


fi_portfolio = FixedIncomePortfolio(bonds=bond_instruments, ref_date=ref_date, di1_connect=None,
                                    bond_market_values=market_values)


print(f'Portfolio Rate {fi_portfolio.rate:.2%}')
print(f'Portfolio Duration {fi_portfolio.duration:.2f}')
print(f'Portfolio Convexity {fi_portfolio.convexity:.2f}')
print(f'Portfolio DV01 {fi_portfolio.dv01:.2f}')
initial_value = fi_portfolio.portfolio_value
print(f'Initial Value: {fi_portfolio.portfolio_value:,.2f}')

db_connect = DBConnect('fhreadonly', 'finquant')
fi_hedge = FixedIncomeHedge(fi_portfolio, db_connect)

duration_hedge = fi_hedge.duration_hedge(0.5)
duration_convexity_hedge = fi_hedge.duration_convexity_hedge(0.5)

Reference Date 2021-02-05 00:00:00
Portfolio Rate 4.47%
Portfolio Duration 1.76
Portfolio Convexity 7.47
Portfolio DV01 17.84
Initial Value: 1,738,213,151.82


In [150]:
# 150 bps increase
hedge_pnl = 0
port_pnl = fi_portfolio.flat_rate_change(rate_change=150, is_bps=True) - initial_value
for code, quantity in duration_hedge.items():
    print(code, quantity)
    hedge_pnl += -quantity*(fi_hedge.di1_price(code, 150, ref_date, True) - fi_hedge.theoretical_price(code, ref_date))

print('Duration Hedge')
print(f'Portfolio PNL {port_pnl:,.2f} Hedge PNL {hedge_pnl:,.2f} Difference {hedge_pnl+port_pnl:,.2f}')

hedge_pnl_2 = 0
for code, quantity in duration_convexity_hedge.items():
    print(code, quantity)
    hedge_pnl_2 += -quantity*(fi_hedge.di1_price(code, 150, ref_date, True) - fi_hedge.theoretical_price(code, ref_date))

print(f'Portfolio PNL {port_pnl:,.2f} Hedge PNL {hedge_pnl_2:,.2f} Difference {hedge_pnl_2+port_pnl:,.2f}')

N22 24229.0
Duration Hedge
Portfolio PNL -44,352,249.04 Hedge PNL 44,995,136.29 Difference 642,887.25
F22 18406.0
F25 5185.0
Portfolio PNL -44,352,249.04 Hedge PNL 44,696,675.23 Difference 344,426.20
